<a href="https://qworld.net" target="_blank" align="left"><img src="../qworld/images/header.jpg"  align="left"></a>
$ \newcommand{\bra}[1]{\langle #1|} $
$ \newcommand{\ket}[1]{|#1\rangle} $
$ \newcommand{\braket}[2]{\langle #1|#2\rangle} $
$ \newcommand{\dot}[2]{ #1 \cdot #2} $
$ \newcommand{\biginner}[2]{\left\langle #1,#2\right\rangle} $
$ \newcommand{\mymatrix}[2]{\left( \begin{array}{#1} #2\end{array} \right)} $
$ \newcommand{\myvector}[1]{\mymatrix{c}{#1}} $
$ \newcommand{\myrvector}[1]{\mymatrix{r}{#1}} $
$ \newcommand{\mypar}[1]{\left( #1 \right)} $
$ \newcommand{\mybigpar}[1]{ \Big( #1 \Big)} $
$ \newcommand{\sqrttwo}{\frac{1}{\sqrt{2}}} $
$ \newcommand{\dsqrttwo}{\dfrac{1}{\sqrt{2}}} $
$ \newcommand{\onehalf}{\frac{1}{2}} $
$ \newcommand{\donehalf}{\dfrac{1}{2}} $
$ \newcommand{\hadamard}{ \mymatrix{rr}{ \sqrttwo & \sqrttwo \\ \sqrttwo & -\sqrttwo }} $
$ \newcommand{\vzero}{\myvector{1\\0}} $
$ \newcommand{\vone}{\myvector{0\\1}} $
$ \newcommand{\stateplus}{\myvector{ \sqrttwo \\  \sqrttwo } } $
$ \newcommand{\stateminus}{ \myrvector{ \sqrttwo \\ -\sqrttwo } } $
$ \newcommand{\myarray}[2]{ \begin{array}{#1}#2\end{array}} $
$ \newcommand{\X}{ \mymatrix{cc}{0 & 1 \\ 1 & 0}  } $
$ \newcommand{\I}{ \mymatrix{rr}{1 & 0 \\ 0 & 1}  } $
$ \newcommand{\Z}{ \mymatrix{rr}{1 & 0 \\ 0 & -1}  } $
$ \newcommand{\Htwo}{ \mymatrix{rrrr}{ \frac{1}{2} & \frac{1}{2} & \frac{1}{2} & \frac{1}{2} \\ \frac{1}{2} & -\frac{1}{2} & \frac{1}{2} & -\frac{1}{2} \\ \frac{1}{2} & \frac{1}{2} & -\frac{1}{2} & -\frac{1}{2} \\ \frac{1}{2} & -\frac{1}{2} & -\frac{1}{2} & \frac{1}{2} } } $
$ \newcommand{\CNOT}{ \mymatrix{cccc}{1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & 1 \\ 0 & 0 & 1 & 0} } $
$ \newcommand{\norm}[1]{ \left\lVert #1 \right\rVert } $
$ \newcommand{\pstate}[1]{ \lceil \mspace{-1mu} #1 \mspace{-1.5mu} \rfloor } $
$ \newcommand{\greenbit}[1] {\mathbf{{\color{green}#1}}} $
$ \newcommand{\bluebit}[1] {\mathbf{{\color{blue}#1}}} $
$ \newcommand{\redbit}[1] {\mathbf{{\color{red}#1}}} $
$ \newcommand{\brownbit}[1] {\mathbf{{\color{brown}#1}}} $
$ \newcommand{\blackbit}[1] {\mathbf{{\color{black}#1}}} $

<font style="font-size:28px;" align="left"><b>Quantum Tomography  </b></font>
<br>
_prepared by Abuzer Yakaryilmaz_
<br><br>
[<img src="../qworld/images/watch_lecture.jpg" align="left">](https://youtu.be/mIEiWCJ6R58)
<br><br><br>

We study a simplified version of quantum tomography here. 

It is similar to learn the bias of a coin by collecting statistics from tossing this coin many times. But, only making measurement may not be enough to make a good guess.

Suppose that you are given 1000 copies of a qubit and your task is to learn the state of this qubit. We use a python class called "unknown_qubit" for doing our quantum experiments. 

Please run the following cell before continuing.

In [6]:
import pennylane as qml
import numpy as np
from math import pi, acos
from random import randrange

class unknown_qubit:
    def __init__(self):
        self.__theta = randrange(18000) / 18000 * pi  # theta ∈ [0, π)
        self.__available_qubits = 1000
        self.__active_qubits = 0
        self.__angle_offset = 0.0  # to simulate Ry rotations
        print(self.__available_qubits, "qubits are created")

    def get_qubits(self, number_of_qubits=None):
        if not isinstance(number_of_qubits, int) or number_of_qubits < 1:
            print("ERROR: Provide a positive integer, e.g., get_qubits(100)")
            return

        if number_of_qubits <= self.__available_qubits:
            self.__active_qubits = number_of_qubits
            self.__available_qubits -= self.__active_qubits
            self.__angle_offset = 0.0  # reset rotations for new batch
            print(f"\nYou have {number_of_qubits} active qubits set to (cos(theta), sin(theta))")
            self.available_qubits()
        else:
            print(f"\nWARNING: You requested {number_of_qubits} qubits, but only {self.__available_qubits} are available!")
            self.available_qubits()

    def rotate_qubits(self, angle=None):
        if not isinstance(angle, (float, int)):
            print("ERROR: rotate_qubits(angle) needs a real number in radians.")
            return

        if self.__active_qubits == 0:
            print("WARNING: No active qubits. Use get_qubits() first.")
            self.available_qubits()
            return

        self.__angle_offset += angle
        print(f"\nYour active qubits are rotated by angle {angle} rad")

    def measure_qubits(self):
        if self.__active_qubits == 0:
            print("WARNING: No active qubits. Use get_qubits() first.")
            self.available_qubits()
            return {}

        shots = self.__active_qubits
        theta_total = self.__theta + self.__angle_offset

        dev = qml.device("default.qubit", wires=1, shots=shots)

        @qml.qnode(dev)
        def circuit():
            qml.RY(2 * theta_total, wires=0)
            return qml.sample(qml.PauliZ(0))

        results = circuit()
        bitstrings = (1 - results) // 2  # convert +1/-1 to 0/1
        counts = {'0': int(np.sum(bitstrings == 0)), '1': int(np.sum(bitstrings == 1))}

        print(f"\nYour {shots} qubits are measured")
        print("counts =", counts)

        self.__active_qubits = 0
        return counts

    def compare_my_guess(self, my_angle):
        if not isinstance(my_angle, (float, int)):
            print("ERROR: compare_my_guess(angle) needs a real number in radians.")
            return

        self.__available_qubits = 0
        diff = abs(my_angle - self.__theta)
        print(f"\n{self.__theta} is the original theta")
        print(f"{my_angle} is your guess")
        print("Difference:", diff / pi * 180, "degrees")
        print("--> Available qubits set to zero; no further experiments allowed.")

    def available_qubits(self):
        print("--> the number of available unused qubit(s) is", self.__available_qubits)


class unknown_qubit:
    
    available_qubit = 1000 -> you get at most 1000 qubit copies
    get_qubits(number_of_qubits) -> you get the specified number of qubits for your experiment
    measure_qubits() -> your qubits are measured and the result is returned as a dictionary variable
                     -> after measurement, these qubits are destroyed
    rotate_qubits(angle) -> your qubits are rotated with the specified angle in radian
    compare_my_guess(my_angle) -> your guess in radian is compared with the real angle

<h3> Task 1 </h3>

You are given 1000 copies of the identical qubits which are in the same quantum state lying in the first or second quadrant of the unit circle. 

This quantum state is represented by an angle $ \theta \in [0,\pi) $, and your task is to guess this angle.

You use the class __unknown_qubit__ and its methods for your experiments. 

_Remark that the measurement outcomes of the quantum states with angles $ \pi \over 3 $ and $ 2 \pi \over 3 $ are identical even though they are different quantum states. Therefore, getting 1000 qubits and then measuring them does not guarantee the correct answer._

Test your solution at least ten times.

__Single experiment__

A direct measument gives us two candidates. We use 900 copies here.

In [10]:
my_experiment = unknown_qubit()

# Estimate using 900 qubits
my_experiment.get_qubits(900)
counts = my_experiment.measure_qubits()

prob_zeros = counts.get('0', 0) / 900
cos_theta = prob_zeros ** 0.5
theta = acos(cos_theta)

theta_first_candidate = theta
theta_second_candidate = pi - theta

print("First candidate:", theta_first_candidate, "rad =", theta_first_candidate * 180 / pi, "deg")
print("Second candidate:", theta_second_candidate, "rad =", theta_second_candidate * 180 / pi, "deg")




1000 qubits are created

You have 900 active qubits set to (cos(theta), sin(theta))
--> the number of available unused qubit(s) is 100

Your 900 qubits are measured
counts = {'0': 454, '1': 446}
First candidate: 0.7809536604232593 rad = 44.74534873754563 deg
Second candidate: 2.3606389931665337 rad = 135.25465126245436 deg


We use remaining 100 copies to test which candidate works better.

For this purpose, we rotate qubits with the first candidate angle in reverse direction. If it is the correct guess, then the new quantum state should be very close to the state $ \ket{0} $ and so we observe only '0's.

In [11]:
# Test which candidate is correct
my_experiment.get_qubits(100)
my_experiment.rotate_qubits(-theta_first_candidate)
counts = my_experiment.measure_qubits()

if counts.get('0', 0) == 100:
    my_guess = theta_first_candidate
else:
    my_guess = theta_second_candidate

my_experiment.compare_my_guess(my_guess)


You have 100 active qubits set to (cos(theta), sin(theta))
--> the number of available unused qubit(s) is 0

Your active qubits are rotated by angle -0.7809536604232593 rad

Your 100 qubits are measured
counts = {'0': 100, '1': 0}

0.7888888219014369 is the original theta
0.7809536604232593 is your guess
Difference: 0.45465126245437004 degrees
--> Available qubits set to zero; no further experiments allowed.


In [12]:
for i in range(10):
    print("\nExperiment", i + 1)
    print("____________")

    my_experiment = unknown_qubit()
    my_experiment.get_qubits(900)
    counts = my_experiment.measure_qubits()

    prob_zeros = counts.get('0', 0) / 900
    cos_theta = prob_zeros ** 0.5
    theta = acos(cos_theta)

    theta_first_candidate = theta
    theta_second_candidate = pi - theta

    my_experiment.get_qubits(100)
    my_experiment.rotate_qubits(-theta_first_candidate)
    counts = my_experiment.measure_qubits()

    if counts.get('0', 0) == 100:
        my_guess = theta_first_candidate
    else:
        my_guess = theta_second_candidate

    my_experiment.compare_my_guess(my_guess)



Experiment 1
____________
1000 qubits are created

You have 900 active qubits set to (cos(theta), sin(theta))
--> the number of available unused qubit(s) is 100

Your 900 qubits are measured
counts = {'0': 9, '1': 891}

You have 100 active qubits set to (cos(theta), sin(theta))
--> the number of available unused qubit(s) is 0

Your active qubits are rotated by angle -1.4706289056333368 rad

Your 100 qubits are measured
counts = {'0': 94, '1': 6}

1.6952383024620923 is the original theta
1.6709637479564563 is your guess
Difference: 1.3908295227332181 degrees
--> Available qubits set to zero; no further experiments allowed.

Experiment 2
____________
1000 qubits are created

You have 900 active qubits set to (cos(theta), sin(theta))
--> the number of available unused qubit(s) is 100

Your 900 qubits are measured
counts = {'0': 878, '1': 22}

You have 100 active qubits set to (cos(theta), sin(theta))
--> the number of available unused qubit(s) is 0

Your active qubits are rotated by angl

<h3> Task 3 (Discussion) </h3>

If the angle in Task 1 is picked in range $ [0,2\pi) $, then can we determine its quadrant correctly?

<h3> Global phase </h3>

Suppose that we have a qubit and its state is either $ \ket{0} $ or $ -\ket{0} $.

Is there any sequence of one-qubit gates such that we can measure different results after applying them?

All one-qubit gates are $ 2 \times 2 $ matrices, and their application is represented by a single matrix: $ A_n \cdot \cdots \cdot A_2 \cdot A_1 = A $.

By linearity, if $ A \ket{0} = \ket{u} $, then $ A (- \ket{0}) = -\ket{u} $. Thus, after measurement, the probabilities of observing state $ \ket{0} $ and state $ \ket{1} $ are the same for $ \ket{u} $ and $ -\ket{u} $. Therefore, we cannot distinguish them.

Even though the states $ \ket{0} $ and $ -\ket{0} $ are different mathematically, they are assumed as identical from the physical point of view. 

The minus sign in front of $ -\ket{0} $ is called as a global phase.

In general, a global phase can be a complex number with magnitude 1.